In [13]:
import pandas as pd
import numpy as np


In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer 
import gensim

In [2]:
df=pd.read_csv('train.En.csv')

In [27]:
df=df[['tweet','sarcastic']]
df.dropna(inplace=True)
print(df.info())

<class 'pandas.core.frame.DataFrame'>
Index: 3467 entries, 0 to 3467
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   tweet      3467 non-null   object
 1   sarcastic  3467 non-null   int64 
dtypes: int64(1), object(1)
memory usage: 81.3+ KB
None


In [5]:
print(df.head())
print(df['sarcastic'].unique())

                                               tweet  sarcastic
0  The only thing I got from college is a caffein...          1
1  I love it when professors draw a big question ...          1
2  Remember the hundred emails from companies whe...          1
3  Today my pop-pop told me I was not “forced” to...          1
4  @VolphanCarol @littlewhitty @mysticalmanatee I...          1
[1 0]


In [6]:
bow_vectorizer = CountVectorizer(max_df=0.90, min_df=2, max_features=1000, stop_words='english')
bow = bow_vectorizer.fit_transform(df['tweet'])

In [25]:
print(bow.shape)

(3467, 1000)


In [7]:
tfidf_vectorizer = TfidfVectorizer(max_df=0.90, min_df=2, max_features=1000, stop_words='english')
tfidf = tfidf_vectorizer.fit_transform(df['tweet'])

In [28]:
print(tfidf.shape)

(3467, 1000)


In [8]:
import nltk
from nltk.tokenize import word_tokenize

nltk.download('punkt')


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\goell\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping tokenizers\punkt.zip.


True

In [10]:
tokenized_tweet = df['tweet'].apply(word_tokenize) # tokenizing 

model_w2v = gensim.models.Word2Vec(
            tokenized_tweet,
            vector_size=200, # desired no. of features/independent variables
            window=5, # context window size
            min_count=2, # Ignores all words with total frequency lower than 2.                                  
            sg = 1, # 1 for skip-gram model
            hs = 0,
            negative = 10, # for negative sampling
            workers= 6, # no.of cores
            seed = 34
) 

model_w2v.train(tokenized_tweet, total_examples= len(df['tweet']), epochs=20)

(1004778, 1529920)

In [16]:
def word_vector(tokens, size):
    vec = np.zeros(size).reshape((1, size))
    count = 0
    for word in tokens:
        try:
            vec += model_w2v.wv[word].reshape((1, size))
            count += 1.
        except KeyError:  # handling the case where the token is not in vocabulary
            continue
    if count != 0:
        vec /= count
    return vec

In [17]:
wordvec_arrays = np.zeros((len(tokenized_tweet), 200)) 
for i in range(len(tokenized_tweet)):
    wordvec_arrays[i,:] = word_vector(tokenized_tweet.iloc[i], 200)
wordvec_df = pd.DataFrame(wordvec_arrays)

In [29]:
print(wordvec_df.shape)

(3467, 200)


In [19]:
from tqdm import tqdm 
tqdm.pandas(desc="progress-bar") 
from gensim.models.doc2vec import TaggedDocument

def add_label(twt):
    output = []
    for i, s in zip(twt.index, twt):
        output.append(TaggedDocument(s, ["tweet_" + str(i)]))
    return output

labeled_tweets = add_label(tokenized_tweet) 

In [21]:
print(labeled_tweets[:5])

[TaggedDocument(words=['The', 'only', 'thing', 'I', 'got', 'from', 'college', 'is', 'a', 'caffeine', 'addiction'], tags=['tweet_0']), TaggedDocument(words=['I', 'love', 'it', 'when', 'professors', 'draw', 'a', 'big', 'question', 'mark', 'next', 'to', 'my', 'answer', 'on', 'an', 'exam', 'because', 'I', '’', 'm', 'always', 'like', 'yeah', 'I', 'don', '’', 't', 'either', '¯\\_', '(', 'ツ', ')', '_/¯'], tags=['tweet_1']), TaggedDocument(words=['Remember', 'the', 'hundred', 'emails', 'from', 'companies', 'when', 'Covid', 'started', 'getting', 'real', '?', 'I', '’', 've', 'gotten', 'three', 'in', 'regards', 'to', 'support', 'for', 'protests', '.', 'And', 'only', '@', 'SavageXFenty', 'shared', 'helpful', 'links', 'and', 'actually', 'said', 'black', 'lives', 'matter', '...', 'we', 'love', 'capitalism', '🥰🙌🏼'], tags=['tweet_2']), TaggedDocument(words=['Today', 'my', 'pop-pop', 'told', 'me', 'I', 'was', 'not', '“', 'forced', '”', 'to', 'go', 'to', 'college', '🙃', 'okay', 'sure', 'sureeee'], tags=

In [32]:
%%time  

#to time the execution of block
model_d2v = gensim.models.Doc2Vec(dm=1, # dm = 1 for ‘distributed memory’ model
                                  dm_mean=1, # dm_mean = 1 for using mean of the context word vectors
                                  vector_size=200, # no. of desired features
                                  window=5, # width of the context window                                  
                                  negative=7, # if > 0 then negative sampling will be used
                                  min_count=5, # Ignores all words with total frequency lower than 5.                                  
                                  workers=32, # no. of cores                                  
                                  alpha=0.1, # learning rate                                  
                                  seed = 23, # for reproducibility
                                 ) 

model_d2v.build_vocab([i for i in tqdm(labeled_tweets)])

model_d2v.train(labeled_tweets, total_examples= len(df['tweet']), epochs=15)

docvec_arrays = np.zeros((len(tokenized_tweet), 200)) 
for i in range(len(df)):
    docvec_arrays[i,:] = model_d2v.dv[i].reshape((1,200))    

docvec_df = pd.DataFrame(docvec_arrays) 

100%|██████████| 3467/3467 [00:00<00:00, 3437742.78it/s]


CPU times: total: 750 ms
Wall time: 4.35 s


In [34]:
print(docvec_df.shape)

(3467, 200)


In [37]:
#70-30 train-test split

xtrain_bow, xtest_bow, ytrain_bow, yvtest_bow = train_test_split(bow, df['sarcastic'], random_state=42, test_size=0.3)
xtrain_tfidf, xtest_tfidf, ytrain_tfidf, yvtest_tfidf = train_test_split(tfidf, df['sarcastic'], random_state=42, test_size=0.3)
xtrain_wordvec, xtest_wordvec, ytrain_wordvec, yvtest_wordvec = train_test_split(wordvec_df, df['sarcastic'], random_state=42, test_size=0.3)
xtrain_docvec, xtest_docvec, ytrain_docvec, yvtest_docvec = train_test_split(docvec_df, df['sarcastic'], random_state=42, test_size=0.3)

In [40]:
from sklearn.svm import SVC
from sklearn import metrics
from sklearn.model_selection import GridSearchCV


In [51]:
def print_statistics(y, y_pred):
    accuracy = metrics.accuracy_score(y, y_pred)
    precision = metrics.precision_score(y, y_pred, average='weighted')
    recall = metrics.recall_score(y, y_pred, average='weighted')
    f_score = metrics.f1_score(y, y_pred, average='weighted')
    print('Accuracy: %.3f\nPrecision: %.3f\nRecall: %.3f\nF_score: %.3f\n'
          % (accuracy, precision, recall, f_score))
    print(metrics.classification_report(y, y_pred))
    return accuracy, precision, recall, f_score

def get_regularization_params(a=-1, b=1, c=3, d=1, e=5):
    reg_range = np.outer(np.logspace(a, b, c), np.array([d, e]))
    reg_range = reg_range.flatten()
    return reg_range

def svm(x_train, y_train, x_test, y_test):
    print("SVM")
    C_range = get_regularization_params(a=-1, b=0, c=2, d=1, e=5)
    gamma_range = get_regularization_params(a=-2, b=-1, c=2, d=1, e=5)
    parameters = {'kernel': ['poly','rbf'], 'C': C_range, 'gamma': gamma_range}
    svm = SVC(class_weight="balanced")

    grid = GridSearchCV(estimator=svm, param_grid=parameters, verbose=0)
    grid.fit(x_train, y_train)
    sorted(grid.cv_results_.keys())
    classifier = grid.best_estimator_
    y_hat = classifier.predict(x_test)
    print_statistics(y_test, y_hat)

# def feature_selection(x_train, y_train, x_test, y_test):
#     print("Feature selection with LinearSVC")
#     model = LinearSVC(C=0.1, penalty='l2')
#     rfe = RFE(model, 5)
#     best_features_model = rfe.fit(x_train, y_train)
#     y_hat = best_features_model.predict(x_test)
#     print_statistics(y_test, y_hat)

In [22]:
xtrain_bow, xtest_bow, ytrain_bow, yvtest_bow = train_test_split(bow, df['sarcastic'], random_state=42, test_size=0.3)
xtrain_tfidf, xtest_tfidf, ytrain_tfidf, yvtest_tfidf = train_test_split(tfidf, df['sarcastic'], random_state=42, test_size=0.3)
xtrain_wordvec, xtest_wordvec, ytrain_wordvec, yvtest_wordvec = train_test_split(wordvec_df, df['sarcastic'], random_state=42, test_size=0.3)
xtrain_docvec, xtest_docvec, ytrain_docvec, yvtest_docvec = train_test_split(docvec_df, df['sarcastic'], random_state=42, test_size=0.3)

In [49]:
svm(xtrain_bow,ytrain_bow,xtest_bow,yvtest_bow)

SVM
Accuracy: 0.724
Precision: 0.663
Recall: 0.724
F_score: 0.610

              precision    recall  f1-score   support

           0       0.72      1.00      0.84       754
           1       0.50      0.00      0.01       287

    accuracy                           0.72      1041
   macro avg       0.61      0.50      0.42      1041
weighted avg       0.66      0.72      0.61      1041



In [50]:
svm(xtrain_tfidf,ytrain_tfidf,xtest_tfidf,yvtest_tfidf)

SVM
Accuracy: 0.714
Precision: 0.640
Recall: 0.714
F_score: 0.636

              precision    recall  f1-score   support

           0       0.73      0.96      0.83       754
           1       0.40      0.08      0.13       287

    accuracy                           0.71      1041
   macro avg       0.57      0.52      0.48      1041
weighted avg       0.64      0.71      0.64      1041



In [52]:
svm(xtrain_wordvec,ytrain_wordvec,xtest_wordvec,yvtest_wordvec)

SVM
Accuracy: 0.619
Precision: 0.624
Recall: 0.619
F_score: 0.621

              precision    recall  f1-score   support

           0       0.74      0.73      0.73       754
           1       0.32      0.33      0.32       287

    accuracy                           0.62      1041
   macro avg       0.53      0.53      0.53      1041
weighted avg       0.62      0.62      0.62      1041



In [53]:
svm(xtrain_docvec,ytrain_docvec,xtest_docvec,yvtest_docvec)

SVM
Accuracy: 0.724
Precision: 0.663
Recall: 0.724
F_score: 0.614

              precision    recall  f1-score   support

           0       0.73      1.00      0.84       754
           1       0.50      0.01      0.02       287

    accuracy                           0.72      1041
   macro avg       0.61      0.50      0.43      1041
weighted avg       0.66      0.72      0.61      1041



In [55]:
import torch
from transformers import BertTokenizer, BertModel

import matplotlib.pyplot as plt

# Load pre-trained model tokenizer (vocabulary)
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertModel.from_pretrained('bert-base-uncased',
           output_hidden_states = True,)


In [56]:
def bert_text_preparation(text, tokenizer):
    marked_text = "[CLS] " + text + " [SEP]"
    tokenized_text = tokenizer.tokenize(marked_text)
    indexed_tokens = tokenizer.convert_tokens_to_ids(tokenized_text)
    segments_ids = [1]*len(indexed_tokens)

    # Convert inputs to PyTorch tensors
    tokens_tensor = torch.tensor([indexed_tokens])
    segments_tensors = torch.tensor([segments_ids])

    return tokenized_text, tokens_tensor, segments_tensors

In [57]:
def get_bert_embeddings(tokens_tensor, segments_tensors, model):
    with torch.no_grad():
        outputs = model(tokens_tensor, segments_tensors)
        hidden_states = outputs[2][1:]

    # Getting embeddings from the final BERT layer
    token_embeddings = hidden_states[-1]
    # Collapsing the tensor into 1-dimension
    token_embeddings = torch.squeeze(token_embeddings, dim=0)

    list_token_embeddings = [token_embed.tolist() for token_embed in token_embeddings]

    return list_token_embeddings

In [64]:
wordvec_arrays = np.zeros((len(df['tweet']), 768)) 
i=0 

for text in df['tweet'].values:
    tokenized_text, tokens_tensor, segments_tensors = bert_text_preparation(text, tokenizer)
    list_token_embeddings = get_bert_embeddings(tokens_tensor, segments_tensors, model)

    def word_vector(size):
      vec = np.zeros(size).reshape((1, size))
      count = 0
      for word in list_token_embeddings:
          try:
              vec += np.array(word).reshape((1, size))
              count += 1.
          except KeyError:  # handling the case where the token is not in vocabulary
              continue
      if count != 0:
          vec /= count
      return vec
    
    wordvec_arrays[i,:] = word_vector(768)
    i=i+1

In [65]:
bert_embeddings = pd.DataFrame(wordvec_arrays)
xtrain_bert, xtest_bert, ytrain_bert, ytest_bert = train_test_split(bert_embeddings, df['sarcastic'], random_state=42, test_size=0.3)

In [66]:
print(bert_embeddings.shape)

(3467, 768)


In [68]:
svm(xtrain_bert, ytrain_bert, xtest_bert, ytest_bert)

SVM
Accuracy: 0.726
Precision: 0.688
Recall: 0.726
F_score: 0.621

              precision    recall  f1-score   support

           0       0.73      0.99      0.84       754
           1       0.58      0.02      0.05       287

    accuracy                           0.73      1041
   macro avg       0.66      0.51      0.44      1041
weighted avg       0.69      0.73      0.62      1041



In [69]:
from sentence_transformers import SentenceTransformer
model = SentenceTransformer('all-MiniLM-L6-v2')

c:\Users\goell\AppData\Local\Programs\Python\Python310\lib\site-packages\huggingface_hub\file_download.py:148: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\goell\.cache\huggingface\hub\models--sentence-transformers--all-MiniLM-L6-v2. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


In [70]:
sent_embeddings = []
for text in df['tweet'].values:
  sentence_embeddings = model.encode(text)
  sent_embeddings.append(sentence_embeddings)

In [71]:
sent_embeddings = pd.DataFrame(sent_embeddings)
xtrain_sent, xtest_sent, ytrain_sent, ytest_sent = train_test_split(sent_embeddings, df['sarcastic'], random_state=42, test_size=0.3)

In [72]:
svm(xtrain_sent, ytrain_sent, xtest_sent, ytest_sent)

SVM
Accuracy: 0.726
Precision: 0.679
Recall: 0.726
F_score: 0.661

              precision    recall  f1-score   support

           0       0.74      0.95      0.83       754
           1       0.51      0.13      0.21       287

    accuracy                           0.73      1041
   macro avg       0.63      0.54      0.52      1041
weighted avg       0.68      0.73      0.66      1041



In [73]:
tokenizer = BertTokenizer.from_pretrained('distilbert-base-uncased')
model = BertModel.from_pretrained('distilbert-base-uncased',
           output_hidden_states = True,)

c:\Users\goell\AppData\Local\Programs\Python\Python310\lib\site-packages\huggingface_hub\file_download.py:148: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\goell\.cache\huggingface\hub\models--distilbert-base-uncased. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)
The tokenizer class you load from this checkpoint is not the same type as the class thi

In [74]:
wordvec_arrays = np.zeros((len(df['tweet']), 768)) 
i=0 

for text in df['tweet'].values:
    tokenized_text, tokens_tensor, segments_tensors = bert_text_preparation(text, tokenizer)
    list_token_embeddings = get_bert_embeddings(tokens_tensor, segments_tensors, model)

    def word_vector(size):
      vec = np.zeros(size).reshape((1, size))
      count = 0
      for word in list_token_embeddings:
          try:
              vec += np.array(word).reshape((1, size))
              count += 1.
          except KeyError:  # handling the case where the token is not in vocabulary
              continue
      if count != 0:
          vec /= count
      return vec
    
    wordvec_arrays[i,:] = word_vector(768)
    i=i+1

In [75]:
distilbert_embeddings = pd.DataFrame(wordvec_arrays)
xtrain_distilbert, xtest_distilbert, ytrain_distilbert, ytest_distilbert = train_test_split(distilbert_embeddings, df['sarcastic'], random_state=42, test_size=0.3)

In [76]:
svm(xtrain_distilbert, ytrain_distilbert, xtest_distilbert, ytest_distilbert)

SVM
Accuracy: 0.723
Precision: 0.524
Recall: 0.723
F_score: 0.608

              precision    recall  f1-score   support

           0       0.72      1.00      0.84       754
           1       0.00      0.00      0.00       287

    accuracy                           0.72      1041
   macro avg       0.36      0.50      0.42      1041
weighted avg       0.52      0.72      0.61      1041



In [77]:
def word_vector_for_sent(list_token_embeddings, size):
      vec = np.zeros(size).reshape((1, size))
      count = 0
      for word in list_token_embeddings:
          try:
              # print(model_w2v[word].shape)
              vec += np.array(word).reshape((1, size))
              count += 1.
          except KeyError:  # handling the case where the token is not in vocabulary
              continue
      if count != 0:
          vec /= count
      return vec

In [79]:
import numpy as np
import torch
from transformers import AutoTokenizer, AutoModel
 
 
def get_word_idx(sent: str, word: str):
    diff_vals=sent.split(" ")
    idxs=np.arange(start=0, stop=len(diff_vals), step=1)

    return idxs


def get_hidden_states(encoded, token_ids_word, model, layers):
    """Push input IDs through model. Stack and sum `layers` (last four by default).
      Select only those subword token outputs that belong to our word of interest
      and average them."""
    with torch.no_grad():
        output = model(**encoded)

    # Get all hidden states
    states = output.hidden_states
    # Stack and sum all requested layers
    output = torch.stack([states[i] for i in layers]).sum(0).squeeze()
    # print(output.shape)
    # Only select the tokens that constitute the requested word
    # word_tokens_output = output[token_ids_word]

    return word_vector_for_sent(output,output.shape[1])


def get_word_vector(sent, idx, tokenizer, model, layers):
    """Get a word vector by first tokenizing the input sentence, getting all token idxs
      that make up the word of interest, and then `get_hidden_states`."""
    encoded = tokenizer(sent, return_tensors="pt")
    # get all token idxs that belong to the word of interest
    token_ids_word = np.where(np.array(encoded.word_ids()) == idx)
    # print(encoded,token_ids_word)

    return get_hidden_states(encoded, token_ids_word, model, layers)


def Run(model_name='bert-base-cased',layers=None):
    # Use last four layers by default
    layers = [-4, -3, -2, -1] if layers is None else layers
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    model = AutoModel.from_pretrained(model_name, output_hidden_states=True)

    embedding=[]
    for text in df['tweet'].values:    
      idx = get_word_idx(text, '')
      word_embedding = get_word_vector(text, idx, tokenizer, model, layers)
      embedding.append(word_embedding)
    
    return embedding

In [80]:
roberta_embeddings = Run("xlm-roberta-base")

c:\Users\goell\AppData\Local\Programs\Python\Python310\lib\site-packages\huggingface_hub\file_download.py:148: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\goell\.cache\huggingface\hub\models--xlm-roberta-base. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)
C:\Users\goell\AppData\Local\Temp\ipykernel_18528\3820518417.py:36: DeprecationWarning: elemen

In [81]:
roberta_embeddings = pd.DataFrame(np.array(roberta_embeddings).squeeze())

In [82]:
xtrain_roberta, xtest_roberta, ytrain_roberta, ytest_roberta = train_test_split(roberta_embeddings, df['sarcastic'], random_state=42, test_size=0.3)

In [83]:
svm(xtrain_roberta, ytrain_roberta, xtest_roberta, ytest_roberta)

SVM
Accuracy: 0.720
Precision: 0.654
Recall: 0.720
F_score: 0.638

              precision    recall  f1-score   support

           0       0.73      0.97      0.83       754
           1       0.45      0.07      0.13       287

    accuracy                           0.72      1041
   macro avg       0.59      0.52      0.48      1041
weighted avg       0.65      0.72      0.64      1041

